In [2]:
import numpy as np
from numba import cuda
import numpy.typing as npt

In [3]:
@cuda.jit
def scatter_kernel(src: npt.NDArray, dst: npt.NDArray, lookup: npt.NDArray):
    tid = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    
    if tid < src.size:
        dst[lookup[tid]] = src[tid] #scatter

In [10]:
src = np.array([10, 20, 30, 40, 50, 60], dtype=np.int32)
lookup = np.array([5, 2, 0, 3, 1, 4], dtype=np.int32)  # Where each value should go
dst = np.zeros_like(src)
    
# Move data to GPU
d_src = cuda.to_device(src)
d_dst = cuda.to_device(dst)
d_lookup = cuda.to_device(lookup)
    
# Configure the grid
block_size = 256
grid_size = (src.size + block_size - 1) // block_size
    
# Run kernel
scatter_kernel[grid_size, block_size](d_src, d_dst, d_lookup)
    
# Copy result back to host
result = d_dst.copy_to_host()
    
# Print results
print("Source array:", src)
print("Lookup table (new position):", lookup)
print("Result array:", result)

Source array: [10 20 30 40 50 60]
Lookup table (new position): [5 2 0 3 1 4]
Result array: [30 50 20 40 60 10]
